In [0]:
import pandas as pd
from datetime import datetime
import numpy as np
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib
import talib

In [0]:

def getCrumb(ticker):
    URL  = "https://finance.yahoo.com/quote/%s/history" % ticker
    import requests 
    import re
    r = requests.get(url = URL) 
    content = str(r.content)
    r1 = re.findall(r"CrumbStore\":\{\"crumb\":\"[A-Za-z0-9.\\\-]+\"",content)[0]
    crumb = r1.replace("CrumbStore\":{\"crumb\":\"","").replace('"',"")
    return crumb,r.cookies


def get_data(symbol, data_type, start_date , end_date):
    try:
        import requests 
        import re
        from io import StringIO
        import os
        import os.path
        
        directory = "drive/My Drive/Investment/data"
        filename = directory+"/"+symbol+"_"+data_type
        if not os.path.exists(directory):
            os.makedirs(directory)
        
        #check cache
        last_saved = 86401
        if os.path.isfile(filename):
            statbuf = os.stat(filename)
            last_saved = datetime.now().timestamp() - statbuf.st_mtime
             
        if(last_saved>(86400)):
            #crawl
            api_url = "https://query1.finance.yahoo.com/v7/finance/download/{0}?period1={1}&period2={2}&interval={3}&events={4}&crumb={5}"
            (crumb, cookies)=getCrumb(symbol)
            api_url = api_url.format(symbol,0,datetime.now().strftime('%s'),"1d", data_type, crumb)
            r = requests.get(url = api_url, cookies = cookies.get_dict(".yahoo.com"))  
            content = str(r.content).replace(",",'","').replace("\\n",'"\r\n"').replace("b\'","").replace("'","")
            res = '"'+content+'"'
            
            #save
            text_file = open(filename, "w")
            n = text_file.write(r.content.decode('utf-8'))
            text_file.close()
        
        #load
        df = pd.read_csv(filename)
        df["Date"]=pd.to_datetime(df['Date'])
        df = df[df["Date"]>=start_date][df["Date"]<=end_date]
        return df  
    except:
        print("Error getting data")

def get_price_data(stock, start_date, end_date):
    print("Getting price data ...")
    price = get_data(stock,"history", start_date, end_date)
    if(price is not None):
        price.rename(columns={'Date':'date','Open':'open','High':'high',
                              'Low':'low','Close':'close','Adj Close':'adjClose','Volume':'vol'}, inplace=True)
        price["date"]=pd.to_datetime(price['date'])
        price = price.set_index(["date"])
        return price
def get_dividend_data(stock, start_date, end_date):
    print("Getting dividend data ...")
    dividend = get_data(stock,"div", start_date, end_date)
    if(dividend is not None):
        dividend.rename(columns={'Date':'date','Dividends':'dividend'}, inplace=True)
        dividend["date"]=pd.to_datetime(dividend['date'])
        dividend = dividend.set_index(["date"])
        return dividend

In [0]:
!pip install talib
def BATCH_ATR(high, low, timeperiod = 14):
    BATCH_ATR = np.full(len(high), np.nan)
    for i in range(0,len(high)):
        if(i<(timeperiod-1)):
            continue
        BATCH_ATR[i]=high[i-(timeperiod-1):i+1].max()-low[i-(timeperiod-1):i+1].min()

    return BATCH_ATR



def SSL(high,low, close, timeperiod=7):
    from talib import EMA
    sma_high = EMA(high,timeperiod )
    sma_low = EMA(low,timeperiod )
    
    ssl_up = np.full(len(close), np.nan)
    ssl_down = np.full(len(close), np.nan)
    lv = np.full(len(close), np.nan)
    lv[timeperiod-0]=1
    for i in range(0,len(close)):
        if(i>=timeperiod):
            if(close[i]<sma_low[i]):
                lv[i] = -1
            else:
                if(close[i]>sma_high[i]):
                    lv[i] = 1
                else:
                    lv[i] = 0
            if(lv[i]==0):
                lv[i] = lv[i-1]
                
            if(lv[i]>0):
                ssl_up[i] = sma_high[i]
                ssl_down[i] = sma_low[i]
            else:
                ssl_up[i] = sma_low[i]
                ssl_down[i] = sma_high[i]
    return ssl_up, ssl_down
def RVI(high, low, close, open, timeperiod = 7):
    rvi = np.full(len(close), np.nan)
    signal = np.full(len(close), np.nan)
    numerator = np.full(len(close), np.nan)
    denominator = np.full(len(close), np.nan)
    for i in range(0,len(close)):
        if(i>=3):
            a = close[i] - open[i]
            b = close[i-1] - open[i-1] 
            c = close[i-2] - open[i-2] 
            d = close[i-3] - open[i-3] 
            e = high[i] - low[i]
            f = high[i-1]  - low[i-1] 
            g = high[i-2]  - low[i-2] 
            h = high[i-3]  - low[i-3]
            numerator[i]=(a+(2*b)+(2*c)+d)/6
            denominator[i]=(e+(2*f)+(2*g)+h)/6
        if(i>=(3+timeperiod)):
            sma_numerator = np.array(numerator)[i-(timeperiod-1):i+1].mean()
            sma_denominator = np.array(denominator)[i-(timeperiod-1):i+1].mean()
            rvi[i] = sma_numerator/sma_denominator
        
        if(i>=6+timeperiod):
            signal[i] = (rvi[i] + (2*rvi[i-1]) + (2*rvi[i-2]) +rvi[i-3])/6
    return rvi,signal

def VWAP(vol, high, low, close):
    cum_vol = vol.cumsum()
    cum_vol_price = (vol * (high + low + close ) /3).cumsum()
    return cum_vol_price / cum_vol

def MATR(high,low, close ,timeperiod=14):
    import statistics
    tr = high - low
    matr = np.full(len(close)+1, np.nan)
    for i in range(0,len(matr)):
        if(i>=timeperiod):
            matr[i]=statistics.median(np.array(tr[i-(timeperiod-1):i+1]))
    return matr

def CLOSE_ATR(high,low, close ,timeperiod=14):
    from talib import SMA
    sma_high = SMA(high,timeperiod )
    sma_low = SMA(low,timeperiod )
    import statistics
    tr = sma_high - sma_low
    matr = np.full(len(close)+1, np.nan)
    for i in range(0,len(matr)):
        if(i>=timeperiod):
            matr[i]=(np.array(tr[i-(timeperiod-1):i+1])).max()
    return matr

def ATR_STOPLOSS(close, high, low, times=3, period=14, early_stop_profit=0.02, stop_early_times=3, repaint = False, median=False, average_high_low=False):
    starting_close = close[period]
    from talib import ATR
    if(average_high_low == False):
        if(median==False):
            atr= ATR(high,low, close ,timeperiod=period)
        else:
            atr=MATR(high,low, close ,timeperiod=period)
    else:
        atr=CLOSE_ATR(high,low, close ,timeperiod=period)
    first = 1
    import math
    atr_trailing_stoploss = np.full(len(close)+1, np.nan)
    for i in range(0,len(close)):
        if(i>0 and math.isnan(atr[i])==False and repaint==True):
            if((close[i-1]<atr_trailing_stoploss[i-1] and close[i]>atr_trailing_stoploss[i])
              or (close[i-1]>atr_trailing_stoploss[i-1] and close[i]<atr_trailing_stoploss[i])):
                starting_close = close[i]
        multiplier = times
        if(i>0 and math.isnan(atr[i])==False):
            if(close[i-1]>atr_trailing_stoploss[i] and close[i]<atr_trailing_stoploss[i]):
                loss = atr[i]*multiplier
                atr_trailing_stoploss[i+1]=close[i]+loss  
                continue
        if(i>0 and math.isnan(atr[i])==False):
            if(close[i-1]<atr_trailing_stoploss[i] and close[i]>atr_trailing_stoploss[i]):
                loss = atr[i]*multiplier
                atr_trailing_stoploss[i+1]=close[i]-loss  
                continue
        if(math.isnan(atr[i])==False):
            loss = atr[i]*multiplier
            atr_trailing_stoploss[i+1]=close[i]-loss
            if(close[i]<atr_trailing_stoploss[i]):
                if(atr_trailing_stoploss[i]>close[i]+loss):
                    '''
                    ratio_to_target = ((close[i]-starting_close)/(starting_close))/early_stop_profit
                    if(((close[i]-starting_close)/(starting_close))<-early_stop_profit):
                        loss = atr[i]*stop_early_times
                    else:
                        if(ratio_to_target<0):
                            loss = atr[i]*(times-((times - stop_early_times)*(-ratio_to_target)))
                        else:
                            loss = atr[i]*multiplier
                    '''
                    atr_trailing_stoploss[i+1]=close[i]+loss
                else:
                    if(math.isnan(atr_trailing_stoploss[i])==False):
                        atr_trailing_stoploss[i+1]=atr_trailing_stoploss[i]
            else:
                if(atr_trailing_stoploss[i]<close[i]-loss):
                    ratio_to_target = ((close[i]-starting_close)/(starting_close))/early_stop_profit
                    if(((close[i]-starting_close)/(starting_close))>early_stop_profit):
                        loss = atr[i]*stop_early_times
                    else:
                        if(ratio_to_target>0):
                            loss = atr[i]*(times-((times - stop_early_times)*ratio_to_target))
                        else:
                            loss = atr[i]*multiplier
                    
                    atr_trailing_stoploss[i+1]=close[i]-loss
                else:
                    if(math.isnan(atr_trailing_stoploss[i])==False):
                        atr_trailing_stoploss[i+1]=atr_trailing_stoploss[i]
    return atr_trailing_stoploss[:-1], atr_trailing_stoploss[-1:]


def BATCH_ATR_STOPLOSS(close, high, low, times=3, period=14, atr_smooth=14, early_stop_profit=0.02, stop_early_times=3, repaint = False, median=False, average_high_low=False):
    starting_close = close[period]
    catch_atr= BATCH_ATR(high,low ,timeperiod=period)
    from talib import EMA
    atr = EMA(catch_atr,timeperiod = atr_smooth)
    first = 1
    import math
    atr_trailing_stoploss = np.full(len(close)+1, np.nan)
    for i in range(0,len(close)):
        if(i>0 and math.isnan(atr[i])==False and repaint==True):
            if((close[i-1]<atr_trailing_stoploss[i-1] and close[i]>atr_trailing_stoploss[i])
              or (close[i-1]>atr_trailing_stoploss[i-1] and close[i]<atr_trailing_stoploss[i])):
                starting_close = close[i]
        multiplier = times
        if(i>0 and math.isnan(atr[i])==False):
            if(close[i-1]>atr_trailing_stoploss[i] and close[i]<atr_trailing_stoploss[i]):
                loss = atr[i]*multiplier
                atr_trailing_stoploss[i+1]=close[i]+loss  
                continue
        if(i>0 and math.isnan(atr[i])==False):
            if(close[i-1]<atr_trailing_stoploss[i] and close[i]>atr_trailing_stoploss[i]):
                loss = atr[i]*multiplier
                atr_trailing_stoploss[i+1]=close[i]-loss  
                continue
        if(math.isnan(atr[i])==False):
            loss = atr[i]*multiplier
            atr_trailing_stoploss[i+1]=close[i]-loss
            if(close[i]<atr_trailing_stoploss[i]):
                if(atr_trailing_stoploss[i]>close[i]+loss):
                    '''
                    ratio_to_target = ((close[i]-starting_close)/(starting_close))/early_stop_profit
                    if(((close[i]-starting_close)/(starting_close))<-early_stop_profit):
                        loss = atr[i]*stop_early_times
                    else:
                        if(ratio_to_target<0):
                            loss = atr[i]*(times-((times - stop_early_times)*(-ratio_to_target)))
                        else:
                            loss = atr[i]*multiplier
                    '''
                    atr_trailing_stoploss[i+1]=close[i]+loss
                else:
                    if(math.isnan(atr_trailing_stoploss[i])==False):
                        atr_trailing_stoploss[i+1]=atr_trailing_stoploss[i]
            else:
                if(atr_trailing_stoploss[i]<close[i]-loss):
                    ratio_to_target = ((close[i]-starting_close)/(starting_close))/early_stop_profit
                    if(((close[i]-starting_close)/(starting_close))>early_stop_profit):
                        loss = atr[i]*stop_early_times
                    else:
                        if(ratio_to_target>0):
                            loss = atr[i]*(times-((times - stop_early_times)*ratio_to_target))
                        else:
                            loss = atr[i]*multiplier
                    
                    atr_trailing_stoploss[i+1]=close[i]-loss
                else:
                    if(math.isnan(atr_trailing_stoploss[i])==False):
                        atr_trailing_stoploss[i+1]=atr_trailing_stoploss[i]
    return atr_trailing_stoploss[:-1], atr_trailing_stoploss[-1:]


def Normalise(listss):
    min_i = min(listss)
    max_i = max(listss)
    if(min_i==max_i):
        return listss
    return [(x-min_i)/(max_i-min_i) for x in listss]
def SLOPE(data):
    #data = np.array(Normalise(data))
    #data = data * (len(data)-1)
    from talib import LINEARREG_ANGLE
    return list(LINEARREG_ANGLE(data,timeperiod = len(data)-1))[-1]
def PRICE_SLOPE(data, period = 14):
    df = data.tail(period)
    first = df.index.start
    last = df.index.stop
    x = df.index.values.reshape(-1, 1) 
    y= df.close.values.reshape(-1, 1) 
    from sklearn.linear_model import LinearRegression
    reg = LinearRegression().fit(x,y)
    first_predicted = reg.predict(np.array([[first]]))[0][0]
    last_predicted = reg.predict(np.array([[last-1]]))[0][0]
    gradient =(last_predicted-first_predicted)/(last-first)
    return gradient


def HEIKIN_ASHI(open_price, high_price, low_price, close_price):
    ha_close = (open_price+ high_price+ low_price+ close_price) / 4
    ha_open = np.full(len(open_price), np.nan)
    ha_high = np.full(len(open_price), np.nan)
    ha_low = np.full(len(open_price), np.nan)
    for i in range(len(open_price)):
        if i == 0:
            ha_open[0]= open_price[0]
        else:
            ha_open[i] = (ha_open[i-1] + ha_close[i-1]) / 2
        ha_high[i] = np.array([high_price[i], ha_open[i], ha_close[i]]).max()
        ha_low[i] = np.array([low_price[i], ha_open[i], ha_close[i]]).min()
    return ha_open, ha_high, ha_low, ha_close

    '''
    if(i>60):
            last_60days_trend_slope = PRICE_SLOPE(data[0:i], period = 60)
            last_60days_max_high = data[0:i].tail(60).high.max()
            last_60days_min_low = data[0:i].tail(60).low.min()
    '''

    '''
        if(i>30):
            last_30days_trend_slope = PRICE_SLOPE(data[0:i], period = 30)
            last_30days_max_high = data[0:i].tail(30).high.max()
            last_30days_min_low = data[0:i].tail(30).low.min()
    '''

    '''
        if(i>14):
            last_14days_trend_slope = PRICE_SLOPE(data[0:i], period = 14)
            last_14days_max_high = data[0:i].tail(14).high.max()
            last_14days_min_low = data[0:i].tail(14).low.min()
            price_perct_14days = (data["close"][i]-last_14days_min_low)/(last_14days_max_high-last_14days_min_low)
    '''

    '''
        max_rsi = data["rsi"][0:i].max()
        min_rsi = data["rsi"][0:i].min()
        rsi_threshold = (max_rsi-min_rsi)*0.02
    '''

def LINEAR_REGRESSION_BOUND(data_close, data_high, data_low, period = 14, skip = 0):
    linreg_upper = np.full(len(data_close), np.nan)
    linreg_lower = np.full(len(data_close), np.nan)
    for i in range(len(data_close)):
        if(i>(period-1+skip)):
            close = np.array(data_close[i-(period)-skip:i-skip])
            high = np.array(data_high[i-(period)-skip:i-skip])
            low = np.array(data_low[i-(period)-skip:i-skip])
            first = 0
            last = period
            x = np.array(list(range(first,last))).reshape(-1, 1) 
            y= np.array(close).reshape(-1, 1) 
            from sklearn.linear_model import LinearRegression
            reg = LinearRegression().fit(x,y)

            max_high1 = 0
            max_high2 = 0
            max_low1 = 0
            max_low2 = 0
            for index in range(len(close)):
                predicted_price = reg.predict(np.array([[index]]))[0][0]
                high_diff = high[index] - predicted_price
                if(max_high1<high_diff):
                    max_high2 = max_high1
                    max_high1 = high_diff
                else:
                    if(max_high2<high_diff):
                        max_high2 = high_diff

                low_diff =  predicted_price - low[index]
                if(max_low1<low_diff):
                    max_low2 = max_low1
                    max_low1 = low_diff
                else:
                    if(max_low2<low_diff):
                        max_low2 = low_diff
            predicted_close = reg.predict(np.array([[len(close)+skip]]))[0][0]
            linreg_upper[i] = predicted_close+((max_high1+max_high2)/2.0)
            linreg_lower[i] = predicted_close-((max_low1+max_low2)/2.0 )
    return linreg_upper, linreg_lower

def PAST_LOW_HIGH(data, period = 14):
    slop_angle = np.full(len(data), np.nan)
    past_high = np.full(len(data), np.nan)
    past_low = np.full(len(data), np.nan)
    for i in range(len(data)):
        if(i>=period):
            slop_angle[i] = SLOPE(data[i-period:i].close)
            past_high[i] = data[0:i-1].tail(period).high.max()
            past_low[i] = data[0:i-1].tail(period).low.min()
    return slop_angle, past_high, past_low
    
    
def STOCHARSI(rsi, timeperiod=14):
    stocha_rsi = np.full(len(rsi), np.nan)
    for i in range(len(rsi)):
        if(i>=14):
            curr_rsi = rsi[i]
            min_rsi = rsi[i-14:i].min()
            max_rsi = rsi[i-14:i].max()
            stocha_rsi[i]=((curr_rsi-min_rsi)/(max_rsi-min_rsi))*100
            if(stocha_rsi[i]>100):
                stocha_rsi[i]=100
            if(stocha_rsi[i]<0):
                stocha_rsi[i]=0
    return stocha_rsi

def KIJUNSEN(high,low, timeperiod=26):
    kijunsen = np.full(len(high), np.nan)
    for i in range(len(high)):
        if(i>=timeperiod):
            kijunsen[i]=(high[i-timeperiod:i].max()+low[i-timeperiod:i].min())/2
    return kijunsen
    

def HMA(close, timeperiod=14):
    import math
    from talib import WMA
    sqrt_period = math.sqrt(timeperiod)
    wma1 = (2*WMA(close, timeperiod = int(timeperiod/2)))-WMA(close, timeperiod = timeperiod)
    return WMA(wma1,timeperiod =int(sqrt_period))
    
def WADDAH_ATTAR_EXPLOSION(close, high, low, sensitive = 150, fast_period=20, slow_period = 40, channel_period = 20, channel_mult = 2, dead_zone=30):
    
    from talib import MACD 
    from talib import BBANDS 
    from talib import ATR 
    from talib import WMA 
    macd, macdsignal, macdhist = MACD(close, fastperiod=fast_period, slowperiod=slow_period, signalperiod=9)
    upperband, middleband, lowerband = BBANDS(close, timeperiod=channel_period, nbdevup=channel_mult, nbdevdn=channel_mult, matype=0)

    ind_trend1 = np.full(len(close), np.nan)
    ind_itrend1 = np.full(len(close), np.nan)
    ind_explo1 = np.full(len(close), np.nan)
    tr = WMA(ATR(high, low, close, 20),3)
    ind_dead = tr*dead_zone / 10
    for i in range(0,len(close)):
        if(i<2):
            continue
        trend1 = (macd[i] - macd[i-1]) * sensitive;
        trend2 = (macd[i-1] - macd[i-2]) * sensitive;
        explo1 = (upperband[i] - lowerband[i])
        #explo2 = (upperband[i-1] - lowerband[i-1])
        
        if(trend1>=0):
            ind_trend1[i]=trend1
            ind_itrend1[i]=0
        if(trend1<0):
            ind_trend1[i]=0
            ind_itrend1[i]=(trend1*-1)
        ind_explo1[i] = explo1
        #print(str(i)+"\t "+str(close[i])+"\t "+str(close[i])+"\t "+str(ind_trend1[i])+"\t"+str(ind_itrend1[i]))
    return ind_trend1, ind_itrend1, ind_explo1, ind_dead


def ASH(close, timeperiod = 9, smooth = 2):
    from talib import WMA 
    bull = np.full(len(close), np.nan)
    bear = np.full(len(close), np.nan)
    bull[1:] = 0.5*(abs(close[1:]-close[:-1])+(close[1:]-close[:-1]))
    bear[1:] = 0.5*(abs(close[1:]-close[:-1])-(close[1:]-close[:-1]))
    
    avgBull = WMA(bull, timeperiod)
    avgBear = WMA(bear, timeperiod)
    
    smoothBull = WMA(avgBull, smooth)
    smoothBear = WMA(avgBear, smooth)
    
    return smoothBull, smoothBear


def RENKO(open, high, low,  close, atr_times = 3, timeperiod = 26):
    renkoHigh = np.full(len(close), np.nan)
    renkoLow = np.full(len(close), np.nan)
    _renkoHigh = np.full(len(close), np.nan)
    _renkoLow = np.full(len(close), np.nan)
    from talib import ATR
    block_size= ATR(high,low, close ,timeperiod=timeperiod)*atr_times
    _counter = np.full(len(close), np.nan)
    
    for i in range(0,len(close)):
        if(i>=timeperiod):
            if(np.isnan(renkoHigh[i-1])):
                if(close[i]>open[i]):
                    renkoHigh[i] = open[i]
                    renkoLow[i] = renkoHigh[i] - block_size[i]
                else:
                    renkoLow[i] = open[i]
                    renkoHigh[i] = renkoLow[i] + block_size[i]
                _renkoHigh[i] = renkoHigh[i]
                _renkoLow[i] = renkoLow[i]
                _counter[i] = 0 
            else:
                renkoHigh[i] = renkoHigh[i-1]
                renkoLow[i] = renkoLow[i-1]
                _renkoHigh[i] = _renkoHigh[i-1]
                _renkoLow[i] = _renkoLow[i-1]
                _counter[i] = _counter[i-1]
                if(close[i]>renkoHigh[i]):
                    gap = int((close[i] - renkoHigh[i])/block_size[i])
                    renkoHigh[i] = renkoHigh[i] + (gap * block_size[i])
                    renkoLow[i] = renkoHigh[i] - block_size[i]
                    if(gap >0):
                        _renkoHigh[i] = renkoHigh[i]
                        _renkoLow[i] = renkoLow[i]
                        if(_renkoHigh[i-1]<_renkoLow[i-1]):
                            _counter[i] = 0
                        else:
                            _counter[i] = _counter[i] + gap
                if(close[i]<renkoLow[i]):
                    gap = int((renkoLow[i] - close[i])/block_size[i])
                    renkoLow[i] = renkoLow[i] - (gap * block_size[i])
                    renkoHigh[i] = renkoLow[i] + block_size[i]
                    if(gap >0):
                        _renkoLow[i]= renkoHigh[i]
                        _renkoHigh[i] = renkoLow[i]
                        if(_renkoHigh[i-1]<_renkoLow[i-1]):
                            _counter[i] = _counter[i] - gap
                        else:
                            _counter[i] = 0
                
    return _renkoHigh, _renkoLow, _counter

def SENKOU(open, high, low,  close, timeperiod1 = 9, timeperiod2 = 26, timeperiod3 = 52):
    
    conversion_line = np.full(len(close), np.nan)
    baseline = np.full(len(close), np.nan)
    senkouA = np.full(len(close), np.nan)
    senkouB= np.full(len(close), np.nan)
    
    
    for i in range(0,len(close)):
        if(i>=timeperiod3-1 and i+timeperiod2<len(close)):
            conversion_line[i] = (close[i-(timeperiod1-1):i+1].max() + close[i-(timeperiod1-1):i+1].min())/2
            baseline[i] = (close[i-(timeperiod2-1):i+1].max() + close[i-(timeperiod2-1):i+1].min())/2
            senkouA[i+timeperiod2] = (conversion_line[i] + baseline[i] )/2
            senkouB[i+timeperiod2] = (close[i-(timeperiod3-1):i+1].max() + close[i-(timeperiod3-1):i+1].min())/2
            
    return senkouA, senkouB
            

  Using cached https://files.pythonhosted.org/packages/1a/80/3492aa3065d7d81a6c51db78920108f84fbe19c4e2740f6bbb83d42f77a4/talib-0.1.1.tar.gz
  ERROR: Failed building wheel for talib
  Running setup.py clean for talib
Failed to build talib
    Running setup.py install for talib ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-ocaich1k/talib/setup.py'"'"'; __file__='"'"'/tmp/pip-install-ocaich1k/talib/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-w1yzft79/install-record.txt --single-version-externally-managed --compile Check the logs for full command output.


In [0]:
class Strategy:
    def get_action(self, data, i, shares_can_buy, shares_can_sell, cash, target, risk_percent, risk_atr_multiplier):
        pass
    
    def set_stoploss(self, data, i, atr_period = 14, atr_multiplier = 3, early_stop_profit = 0.02, stop_early_times=1, danger_stop_atr=5):
        data.loc[i+1:,("stoploss")] = ATR_STOPLOSS(close = np.array(data["close"][i-atr_period:]), 
                                                         high = np.array(data["high"][i-atr_period:]),
                                                         low = np.array(data["low"][i-atr_period:]), 
                                     times=atr_multiplier, period=atr_period, early_stop_profit=early_stop_profit,stop_early_times=stop_early_times)[0][atr_period+1:]
        '''
        
       
        data.loc[i+1:,("stoploss_danger")] = ATR_STOPLOSS(close = np.array(data["close"][i-atr_period:]), 
                                                         high = np.array(data["high"][i-atr_period:]),
                                                         low = np.array(data["low"][i-atr_period:]), 
                                     times=danger_stop_atr, period=atr_period, early_stop_profit=early_stop_profit,stop_early_times=danger_stop_atr)[0][atr_period+1:]
        '''
        return data
    import numpy as np
    def set_tp_sl(self, data, i, take_profit=np.nan, stop_loss=np.nan):
        data.loc[i+1:,("take_profit")] = take_profit
        data.loc[i+1:,("stoploss_fix")] = stop_loss        
        return data
    
    def get_col_to_plot(self):
        pass

    def preprocess(self, data):
        pass

def back_test(strategy, data, cash = 100000,transaction_fee = 0.01, assets={}, min_transaction = 1000, target=0.02, risk_percent = 0.01, risk_atr_multiplier = 3):
    longs = {}
    shorts = {}
    init_cash = cash
    bought_at = 100000
    bought_at_list = []
    loss_trade_count = 0
    win_trade_count = 0
    win_percent = 0
    loss_percent = 0
    
    
    process_col(data,"atr_risk",14)
    strategy.preprocess(data)
    
    for i,row in data.iterrows():
        shares_can_buy = int(cash/(row.close*(1+transaction_fee)))
        shares_can_sell=0
        if row.stock in assets:
            shares_can_sell = assets[row.stock]
        #Decision
        recommended_shares = -1
        shares_held = 0
        if(i>0):
            current_assets = cash
            if(data.iloc[i].stock in assets):
                shares_held = assets[data.iloc[i-1].stock]
                current_assets = shares_held*data.iloc[i-1].close + cash
            risk_atr = data["atr_risk_14"][i-1]
            last_close = data["close"][i-1]
            import math
            if(math.isnan(risk_atr)==False):
                risk_atr =  risk_atr * risk_atr_multiplier
                risk_amt = current_assets * risk_percent
                if(risk_atr!=0):
                    recommended_shares = int(risk_amt/risk_atr)
                    if(shares_can_buy>0):
                        if(shares_held+shares_can_buy>recommended_shares):
                            shares_can_buy = int(recommended_shares-shares_held)            
                else:
                    continue
            else:
                continue
        #if(recommended_shares<=0 or recommended_shares*row.close*(1+transaction_fee)<min_transaction):
        #    continue
        action , price_point, data = strategy.get_action(data, i, shares_can_buy, shares_can_sell, target, risk_percent, risk_atr_multiplier)
        #Execution
        if(action=="BUY"):
            if(shares_can_buy>0):
                cost = shares_can_buy*price_point*(1+transaction_fee)
                cash_left = cash-cost
                if row.stock not in assets:
                    assets[row.stock]=0
                assets[row.stock] = assets[row.stock] + shares_can_buy
                cash = cash_left
                longs[row.date] = price_point
                bought_at = price_point
                bought_at_list = bought_at_list + [price_point]
                #print("BUY : +" +str(shares_can_buy) +" @"+str(price_point))
        if(action=="SELL"):
            if(shares_can_sell>0):
                asset = shares_can_sell * price_point
                cash = cash + asset*(1-transaction_fee)
                assets.clear()
                shorts[row.date] = price_point
                #trade counter
                for trade in bought_at_list:
                    if(trade>price_point):
                        loss_trade_count = loss_trade_count +1
                        loss_percent = loss_percent + ((price_point-trade)/trade)
                    else:
                        win_trade_count = win_trade_count +1
                        win_percent = win_percent + ((price_point-trade)/trade)
                bought_at_list = []
                #print("SELL : -" +str(shares_can_sell) +" @"+str(price_point))
        if(action=="STOPPED"):
            if(shares_can_sell>0):
                asset = shares_can_sell * price_point
                cash = cash + asset*(1-transaction_fee)
                assets.clear()
                shorts[row.date] = price_point
                #trade counter
                for trade in bought_at_list:
                    if(trade>price_point):
                        loss_trade_count = loss_trade_count +1
                        loss_percent = loss_percent + ((price_point-trade)/trade)
                    else:
                        win_trade_count = win_trade_count +1
                        win_percent = win_percent + ((price_point-trade)/trade)
                bought_at_list = []
                #print("STOPPED : -" +str(shares_can_sell) +" @"+str(price_point))
    final_cash = cash
    if(data.iloc[-1].stock in assets):
        final_cash = assets[data.iloc[-1].stock]*data.iloc[-1].close + cash
    assets.clear()
    growth =((final_cash-init_cash)/init_cash)*100
    print("\tprofit: "+str(round(growth,2))+"%"+"\t"+"asset: "+str(final_cash))
    print("\t\t"+"win count: "+str(win_trade_count)+"\t"+"loss count: "+str(loss_trade_count))
    print("\t\t"+"win percent: "+str(win_percent)+"\t"+"loss percent: "+str(loss_percent))
    return round(growth,2), longs, shorts,  win_trade_count, loss_trade_count, win_percent, loss_percent, final_cash


In [0]:
def lookback(i, data, c3_co1, c3_co2, period):
    c3_signal=[-1]*period
    c3_agree=[False]*period
    
    for p in range(0,period):
        if(data[c3_co1][i-p]>data[c3_co2][i-p]):
            c3_agree[p] = True
            for d in range(0,period):
                if(data[c3_co1][(i-p)-d]>data[c3_co2][(i-p)-d]
                  and data[c3_co1][(i-p)-(d+1)]<data[c3_co2][(i-p)-(d+1)]):
                    c3_signal[p] = d
                    break
    return c3_signal, c3_agree

def process_col(data, col="", *argv):
    params = '_'.join(str(x) for x in argv)
    if(col+"_"+params in data.columns):
        return
    
    if(col=="zero"):
        data['zero'] = np.full(len(data), 0)
       
    if(col=="atr_risk"):
        from talib import ATR
        data["atr_risk_"+params]= ATR(data['high'].values, data['low'].values, data['close'].values ,timeperiod=argv[0])
    
    if(col=="macd"):
        from talib import MACD
        data['macd_'+params], data['macd_signal_'+params], data['macd_hist_'+params] = MACD(data['close'], fastperiod=argv[0], slowperiod=argv[1], signalperiod=argv[2])
    if(col=="hamacd"):
        from talib import MACD
        data['hamacd_'+params], data['hamacd_signal_'+params], data['hamacd_hist_'+params] = MACD(data['ha_close'], fastperiod=argv[0], slowperiod=argv[1], signalperiod=argv[2])


    if(col=="rsi"):
        from talib import RSI
        data['rsi_'+params] = RSI(data['close'].values, timeperiod=argv[0])

    if(col=="adx"):
        from talib import ADX
        data['adx_'+params] =  ADX(data['high'].values, data['low'].values, data['close'].values, timeperiod=argv[0])

    if(col=="kijunsen"):
        data['kijunsen_'+params] = KIJUNSEN(data['high'],data['low'], timeperiod=argv[0])
        
    if(col=="ema"):
        from talib import EMA
        data['ema_'+params] = EMA(data[argv[0]], timeperiod=argv[1])
        
    if(col=="ema_shift"):
        from talib import EMA
        data['ema_shift_'+params] = EMA(data[argv[0]], timeperiod=argv[1]).shift(argv[2], axis = 0) 
        
    if(col=="sma"):
        from talib import SMA
        data['sma_'+params] = SMA(data[argv[0]], timeperiod=argv[1])
    if(col=="sma_shift"):
        from talib import SMA
        data['sma_shift_'+params] = SMA(data[argv[0]], timeperiod=argv[1]).shift(argv[2], axis = 0) 
    if(col=="wma"):
        from talib import WMA
        data['wma_'+params] = WMA(data[argv[0]], timeperiod=argv[1])
        
    if(col=="hma"):
        data['hma_'+params] = HMA(data[argv[0]], timeperiod=argv[1])
        
    if(col=="linearreg"):
        from talib import LINEARREG_ANGLE
        data['linearreg_'+params] = LINEARREG_ANGLE(data[argv[0]], timeperiod=argv[1])
        
    if(col=="linearreg"):
        from talib import LINEARREG_ANGLE
        data['linearreg_'+params] = LINEARREG_ANGLE(data[argv[0]], timeperiod=argv[1])
    
    if(col=="atr_stoploss"):
        from talib import LINEARREG_ANGLE
        data['atr_stoploss_'+params] = ATR_STOPLOSS(close = data.close.values, 
                                                   high = data.high.values, 
                                                   low = data.low.values, 
                                 times=argv[0], stop_early_times=argv[1], early_stop_profit=argv[2], period=argv[3], repaint=True)[0]
    if(col=="batch_atr_stoploss"):
        from talib import LINEARREG_ANGLE
        data['batch_atr_stoploss_'+params] = BATCH_ATR_STOPLOSS(close = data.close.values, 
                                                   high = data.high.values, 
                                                   low = data.low.values, 
                                 times=argv[0], stop_early_times=argv[1], early_stop_profit=argv[2], period=argv[3], atr_smooth=argv[4], repaint=True)[0]

    if(col=="batch_atr"):
        data['batch_atr_'+params] = BATCH_ATR(data['high'].values, data['low'].values, timeperiod=argv[0])
    

    if(col=="atr"):
        from talib import ATR
        data['atr_'+params] = ATR(data['high'].values, data['low'].values, data['close'].values ,timeperiod=argv[0])
    
    
    if(col=="ssl"):
        data["ssl_up_"+params],data["ssl_down_"+params]= SSL(data['high'].values, data['low'].values, data['close'].values ,timeperiod=argv[0])
    
    
    if(col=="ha"):
        data["ha_open"],data["ha_high"],data["ha_low"],data["ha_close"] = HEIKIN_ASHI(data['open'].values, data['high'].values, data['low'].values, data['close'].values)

    if(col=="rvi"):
        data["rvi_"+params],data["rvi_signal_"+params]= RVI(data['high'].values, data['low'].values, data['close'].values, data['open'].values,timeperiod=argv[0])

    if(col=="waddah"):
        data["waddah_bull_"+params],data["waddah_bear_"+params],data["waddah_explo_"+params],data["waddah_dead_"+params]= WADDAH_ATTAR_EXPLOSION(data['close'].values,data['high'].values,data['low'].values, sensitive = argv[0] , fast_period= argv[1], slow_period =  argv[2], channel_period =  argv[3], channel_mult =  argv[4], dead_zone= argv[5])
    
    if(col=="ash"):
        data["ASH_bull_"+params],data["ASH_bear_"+params]= ASH(data['close'].values, timeperiod=argv[0], smooth =argv[1])

    if(col=="hline"):
        data["hline_"+params]= argv[0]
        
        
    if(col=="stocharsi"):
        from talib import RSI
        data["stocharsi_"+params]= STOCHARSI(RSI(data['close'].values, timeperiod=argv[0]),timeperiod=argv[1])
        
    if(col=="stocha"):
        data["stocha_"+params]= STOCHARSI(data['close'].values,timeperiod=argv[1])
        
    if(col=="renko"):
        data["renko_high_"+params], data["renko_low_"+params], data["renko_count_"+params]= RENKO(data['open'].values,data['high'].values,data['low'].values,data['close'].values,atr_times=argv[0],timeperiod=argv[1])
          
    if(col=="senkou"):
        data["senkouA_"+params], data["senkouB_"+params]= SENKOU(data['open'].values,data['high'].values,data['low'].values,data['close'].values,timeperiod1=argv[0],timeperiod2=argv[1],timeperiod3=argv[2])
        
    if(col=="bband"):
        from talib import BBANDS
        data["bband_high_"+params], data["bband_mid_"+params], data["bband_low_"+params] = BBANDS(data['close'].values, timeperiod=argv[0], nbdevup=argv[1], nbdevdn=argv[2], matype=0)
        
def name_col(col="", *argv):
    params = '_'.join(str(x) for x in argv)
    return col+"_"+params

In [0]:
def plot_chart(stock_prices, result, stock, strategy):
    ######Plot
    ######
    growth, longs, shorts,  win_trade_count, loss_trade_count, win_percent, loss_percent, final_cash=result
    import plotly.graph_objs as go 
    from datetime import datetime
    from ipywidgets import interact, interactive, fixed, interact_manual
    import ipywidgets as widgets
    from plotly.subplots import make_subplots
    # Make sure dates are in ascending order
    # We need this for slicing in the callback below
    data = stock_prices
    df = data.set_index('date')
    fig = go.FigureWidget(make_subplots(rows=5, cols=1))
    
    
    
    
    fig.add_trace(
        go.Scattergl(x=list(df.index), y=list(df['close']), line = dict(color='black'),
        name = "close"),
        row=1, col=1
    )
    if('stoploss' in df.columns):
        fig.add_trace(
            go.Scattergl(x=list(df.index), y=list(df['stoploss']), line = dict(color='red', width=2, dash='dash'),
            name = "stoploss"),
            row=1, col=1
        )
    if('stoploss_fix' in df.columns):
        fig.add_trace(
            go.Scattergl(x=list(df.index), y=list(df['stoploss_fix']), line = dict(color='firebrick', width=2, dash='dash'),
            name = "stoploss_fix"),
            row=1, col=1
        )
        
    if('stoploss_danger' in df.columns):
        fig.add_trace(
            go.Scattergl(x=list(df.index), y=list(df['stoploss_danger']), line = dict(color='firebrick', width=2, dash='dash'),
            name = "stoploss_danger"),
            row=1, col=1
        )
    if('free_line' in df.columns):
        fig.add_trace(
            go.Scattergl(x=list(df.index), y=list(df['free_line']), line = dict(color='blue', width=2, dash='dot'),
            name = "free_line"),
            row=1, col=1
        )
    if('free_line2' in df.columns):
        fig.add_trace(
            go.Scattergl(x=list(df.index), y=list(df['free_line2']), line = dict(color='green', width=2, dash='dot'),
            name = "free_line2"),
            row=1, col=1
        )
    
    if('take_profit' in df.columns):
        fig.add_trace(
            go.Scattergl(x=list(df.index), y=list(df['take_profit']), line = dict(color='blue', width=2, dash='dash'),
            name = "take_profit"),
            row=1, col=1
        )
        
    fig.add_trace(
        go.Scattergl(x=[i for i in shorts], y=[shorts[i] for i in shorts],
        name = "Short", mode="markers", marker=dict(color="red", size=5)),
        row=1, col=1
    )
    fig.add_trace(
        go.Scattergl(x=[i for i in longs], y=[longs[i] for i in longs],
        name = "Long", mode="markers", marker=dict(color="green", size=5)),
        row=1, col=1
    )
    
    plot1,plot2,plot3,plot4,plot5 = strategy.get_col_to_plot()
    
    for plott in plot1:
        p = plott[0]
        marker = plott[1]
        fig.add_trace(
            go.Scattergl(x=list(df.index), y=list(df[p]),
            name = p, mode=marker, marker=dict(size=4)),
            row=1, col=1
        )
    for plott in plot2:
        p = plott[0]
        marker = plott[1]
        fig.add_trace(
            go.Scattergl(x=list(df.index), y=list(df[p]),
            name = p, mode=marker, marker=dict(size=4)),
            row=2, col=1
        )
    for plott in plot3:
        p = plott[0]
        marker = plott[1]
        fig.add_trace(
            go.Scattergl(x=list(df.index), y=list(df[p]),
            name = p, mode=marker, marker=dict(size=4)),
            row=3, col=1
        )
    for plott in plot4:
        p = plott[0]
        marker = plott[1]
        fig.add_trace(
            go.Scattergl(x=list(df.index), y=list(df[p]),
            name = p, mode=marker, marker=dict(size=4)),
            row=4, col=1
        )
    for plott in plot5:
        p = plott[0]
        marker = plott[1]
        fig.add_trace(
            go.Scattergl(x=list(df.index), y=list(df[p]),
            name = p, mode=marker, marker=dict(size=4)),
            row=5, col=1
        )
    fig.layout.xaxis=dict(
            anchor='x',
            rangeselector=dict(
                buttons=list([
                    dict(count=1,
                         label='1m',
                         step='month',
                         stepmode='backward'),
                    dict(count=6,
                         label='6m',
                         step='month',
                         stepmode='backward'),
                    dict(count=1,
                        label='YTD',
                        step='year',
                        stepmode='todate'),
                    dict(count=1,
                        label='1y',
                        step='year',
                        stepmode='backward'),
                    dict(step='all')
                ])
            ),
            type='date'
        )
    fig.layout.yaxis=dict(
            domain=[0.7, 1]
        )
    fig.layout.yaxis2=dict(
            domain=[0.525, 0.69]
        )
    fig.layout.yaxis3=dict(
            domain=[0.35, 0.524]
        )
    fig.layout.yaxis4=dict(
            domain=[0.175, 0.34]
        )
    fig.layout.yaxis5=dict(
            domain=[0.0, 0.174]
        )
    #fig.layout.yaxis2.range=[0,100]
    #fig.layout.yaxis2.tickvals=[0,20,80,100]
    fig['layout'].update(height=900, width=1200) 
    
    import time
    import datetime
    def zoom(layout, xrange):
        try:
            fig.layout.xaxis2.range = fig.layout.xaxis.range
            fig.layout.xaxis3.range = fig.layout.xaxis.range
            fig.layout.xaxis4.range = fig.layout.xaxis.range
            fig.layout.xaxis5.range = fig.layout.xaxis.range
            in_view = df.loc[fig.layout.xaxis.range[0]:fig.layout.xaxis.range[1]]
            padding = 0.1
            
            from_price = in_view.iloc[0].close
            to_price = in_view.iloc[-1].close
            growth = (to_price-from_price)/from_price*100.0
            
            max_val_y = in_view["close"].max()
            min_val_y =  in_view["close"].min()
            axis_diff = max_val_y-min_val_y
            fig.layout.yaxis.range = [min_val_y - (axis_diff*padding), max_val_y + (axis_diff*padding)]
            
            if(len(plot2)>0):
                max_val_y = in_view[plot2[0][0]].max()
                min_val_y =  in_view[plot2[0][0]].min()
                axis_diff = max_val_y-min_val_y
                fig.layout.yaxis2.range = [min_val_y - (axis_diff*padding), max_val_y + (axis_diff*padding)]
         
            if(len(plot3)>0):
                max_val_y = in_view[plot3[0][0]].max()
                min_val_y =  in_view[plot3[0][0]].min()
                axis_diff = max_val_y-min_val_y
                fig.layout.yaxis3.range = [min_val_y - (axis_diff*padding), max_val_y + (axis_diff*padding)]
         
            if(len(plot4)>0):
                max_val_y = in_view[plot4[0][0]].max()
                min_val_y =  in_view[plot4[0][0]].min()
                axis_diff = max_val_y-min_val_y
                fig.layout.yaxis4.range = [min_val_y - (axis_diff*padding), max_val_y + (axis_diff*padding)]
                
            if(len(plot5)>0):
                max_val_y = in_view[plot5[0][0]].max()
                min_val_y =  in_view[plot5[0][0]].min()
                axis_diff = max_val_y-min_val_y
                fig.layout.yaxis5.range = [min_val_y - (axis_diff*padding), max_val_y + (axis_diff*padding)]
         
            fig['layout'].update(title=symbol+" ("+str(round(growth,2))+"%)") 
        except Exception as e:
            None
    fig.layout.on_change(zoom, 'xaxis')
    return fig

In [0]:
class Strategy1(Strategy):
    def __init__(self,*arg):
        if(len(arg)==0):
            self.arg = [26,12,26,20]
        else:
            self.arg= arg
    def preprocess(self, data):
        
        self.kijunsen_param = 26
        self.ssl_param_1 = 50
        self.waddah_param1 = 150 #sensitive
        self.waddah_param2 = 12 #fast_period
        self.waddah_param3 = 26 #slow_period
        self.waddah_param4 = 26 #channel_period
        self.waddah_param5 = 2 #channel_mult
        self.waddah_param6 = 30 #dead_zone
        self.rvi_param1 = 12 #timeperiod
        #self.hma_smooth_rvi = 4
        self.adx_param1 = 14 #timeperiod
        self.adx_sma_param1 = 100
        self.confirmation_delay = 4
        self.sma_shift_param_1 = 5
        self.sma_shift_param_2 = 3
        
        self.ema_param_1 = 7
        self.ema_param_2 = 12
        self.ema_param_3 = 26
        self.ema_param_4 = 50
        self.sma_param_1 = 5
        self.sma_param_2 = 12
        self.sma_param_3 = 26
        self.sma_param_4 = 50
        self.hma_param_1 = 5
        self.hma_param_2 = 26
        self.hma_param_3 = 50
        self.hma_param_4 = 100
        self.hline_param_20 = 25
        self.macd_param_1 = 12
        self.macd_param_2 = 26
        self.macd_param_3 = 9
        self.stocharsi_param1 = 14
        self.stocharsi_param2 = 14
        self.batch_atr_param = 50
        self.batch_atr_smooth_param = 5
        self.atr_param_1 = 50
        self.atr_stoploss_1_param_1 = 1
        self.atr_stoploss_1_param_2 = 0.05
        self.atr_stoploss_1_param_3 = 0.01
        self.atr_stoploss_1_param_4 = 26
        self.atr_smooth_param = 3
        self.early_stop = 3
        self.ash_param1 = 18
        self.ash_param2 = 5
        
        self.renko_atr = 1
        self.renko_timeperiod = 50
        self.renko2_atr = 1
        self.renko2_timeperiod = 26
        self.senkou_param1 = 3
        self.senkou_param2 = 13
        self.senkou_param3 = 26
        self.senkou2_param1 = 9
        self.senkou2_param2 = 26
        self.senkou2_param3 = 52
        self.bband_param_1=14
        self.bband_param_2=2
        self.bband_param_3=2
        
        self.hamacd_param_1 = 12
        self.hamacd_param_2 = 26
        self.hamacd_param_3 = 9
        
        self.stocharsi = name_col("stocharsi",self.stocharsi_param1,self.stocharsi_param2)
        self.batch_atr = name_col("batch_atr",self.batch_atr_param)
        self.batch_atr_wma = name_col("wma",self.batch_atr,self.batch_atr_smooth_param)
        
        self.hline_20 = name_col("hline",self.hline_param_20)
        self.atr = name_col("atr",self.atr_param_1)
        self.adx = name_col("adx",self.adx_param1)
        self.adx_sma = name_col("sma",self.adx,self.adx_sma_param1)
        self.kijunsen = name_col("kijunsen",self.kijunsen_param)
        self.ssl_up = name_col("ssl_up",self.ssl_param_1)
        self.ssl_down = name_col("ssl_down",self.ssl_param_1)
        self.ASH_bull = name_col("ASH_bull",self.ash_param1,self.ash_param2)
        self.ASH_bear = name_col("ASH_bear",self.ash_param1,self.ash_param2)
        self.waddah_bull  = name_col("waddah_bull",self.waddah_param1,self.waddah_param2,self.waddah_param3,self.waddah_param4,self.waddah_param5,self.waddah_param6)
        self.waddah_bear = name_col("waddah_bear",self.waddah_param1,self.waddah_param2,self.waddah_param3,self.waddah_param4,self.waddah_param5,self.waddah_param6)
        self.waddah_explo = name_col("waddah_explo",self.waddah_param1,self.waddah_param2,self.waddah_param3,self.waddah_param4,self.waddah_param5,self.waddah_param6)
        self.waddah_dead = name_col("waddah_dead",self.waddah_param1,self.waddah_param2,self.waddah_param3,self.waddah_param4,self.waddah_param5,self.waddah_param6)
        self.rvi = name_col("rvi",self.rvi_param1)
        self.rvi_signal = name_col("rvi_signal",self.rvi_param1)
        #self.rvi_hma = name_col("hma",self.rvi,self.hma_smooth_rvi)
        #self.rvi_signal_hma = name_col("hma",self.rvi_signal,self.hma_smooth_rvi)
        self.ema_1  = name_col("ema","close",self.ema_param_1)
        self.ema_2  = name_col("ema","close",self.ema_param_2)
        self.ema_3  = name_col("ema","close",self.ema_param_3)
        self.ema_4  = name_col("ema","close",self.ema_param_4)
        self.sma_shift_1 = name_col("sma_shift","close",self.sma_shift_param_1,self.sma_shift_param_2)
        self.sma_1 = name_col("sma","close",self.sma_param_1)
        self.sma_2 = name_col("sma","close",self.sma_param_2)
        self.sma_3 = name_col("sma","close",self.sma_param_3)
        self.sma_4 = name_col("sma","close",self.sma_param_4)
        self.hma_1 = name_col("hma","close",self.hma_param_1)
        self.hma_2 = name_col("hma","close",self.hma_param_2)
        self.hma_3 = name_col("hma","close",self.hma_param_3)
        self.hma_4 = name_col("hma","close",self.hma_param_4)
        self.bband_high = name_col("bband_high",self.bband_param_1, self.bband_param_2, self.bband_param_3)
        self.bband_mid = name_col("bband_mid",self.bband_param_1, self.bband_param_2, self.bband_param_3)
        self.bband_low = name_col("bband_low",self.bband_param_1, self.bband_param_2, self.bband_param_3)
    
        self.senkouA = name_col("senkouA",self.senkou_param1,self.senkou_param2,self.senkou_param3)
        self.senkouB = name_col("senkouB",self.senkou_param1,self.senkou_param2,self.senkou_param3)
        self.senkouA2 = name_col("senkouA",self.senkou2_param1,self.senkou2_param2,self.senkou2_param3)
        self.senkouB2 = name_col("senkouB",self.senkou2_param1,self.senkou2_param2,self.senkou2_param3)
        
        self.renkoHigh = name_col("renko_high",self.renko_atr,self.renko_timeperiod)
        self.renkoLow = name_col("renko_low",self.renko_atr,self.renko_timeperiod)
        self.renkoCount = name_col("renko_count",self.renko_atr,self.renko_timeperiod)
        self.renko2High = name_col("renko_high",self.renko2_atr,self.renko2_timeperiod)
        self.renko2Low = name_col("renko_low",self.renko2_atr,self.renko2_timeperiod)
        self.renko2Count = name_col("renko_count",self.renko2_atr,self.renko2_timeperiod)
 
        self.macd = name_col("macd",self.macd_param_1,self.macd_param_2,self.macd_param_3)
        self.macd_signal = name_col("macd_signal",self.macd_param_1,self.macd_param_2,self.macd_param_3)
        self.macd_hist = name_col("macd_hist",self.macd_param_1,self.macd_param_2,self.macd_param_3)
        
        
        self.ha_open = "ha_open"
        self.ha_close = "ha_close"
        self.ha_high = "ha_high"
        self.ha_low ="ha_low"
        
        self.hamacd = name_col("hamacd",self.hamacd_param_1,self.hamacd_param_2,self.hamacd_param_3)
        self.hamacd_signal = name_col("hamacd_signal",self.hamacd_param_1,self.hamacd_param_2,self.hamacd_param_3)
        self.hamacd_hist = name_col("hamacd_hist",self.hamacd_param_1,self.hamacd_param_2,self.hamacd_param_3)
        
        self.atr_stoploss_1 = name_col("batch_atr_stoploss",self.atr_stoploss_1_param_1,
                              self.atr_stoploss_1_param_2,self.atr_stoploss_1_param_3,
                              self.atr_stoploss_1_param_4,self.atr_smooth_param)
        
        
        process_col(data,"ha")
        process_col(data,"bband",self.bband_param_1, self.bband_param_2, self.bband_param_3)
        process_col(data,"sma_shift","close",self.sma_shift_param_1,self.sma_shift_param_2)
        process_col(data,"senkou",self.senkou_param1,self.senkou_param2,self.senkou_param3)
        process_col(data,"senkou",self.senkou2_param1,self.senkou2_param2,self.senkou2_param3)
        process_col(data,"renko",self.renko_atr,self.renko_timeperiod)
        process_col(data,"renko",self.renko2_atr,self.renko2_timeperiod)
        process_col(data,"stocharsi",self.stocharsi_param1,self.stocharsi_param2)
        process_col(data,"batch_atr",self.batch_atr_param)
        process_col(data,"wma",self.batch_atr,self.batch_atr_smooth_param)
        
        process_col(data,"hline",self.hline_param_20)
        process_col(data,"kijunsen",self.kijunsen_param)
        process_col(data,"atr",self.atr_param_1)
        process_col(data,"ssl",self.ssl_param_1)
        process_col(data,"waddah",self.waddah_param1,self.waddah_param2,self.waddah_param3,self.waddah_param4,self.waddah_param5,self.waddah_param6)
        process_col(data,"ash",self.ash_param1,self.ash_param2)
        process_col(data,"rvi",self.rvi_param1)
        process_col(data,"adx",self.adx_param1)
        process_col(data,"sma",self.adx,self.adx_sma_param1)
        #process_col(data,"hma",self.rvi,self.hma_smooth_rvi)
        #process_col(data,"hma",self.rvi_signal,self.hma_smooth_rvi)
        
        process_col(data,"sma","close",self.sma_param_1)
        process_col(data,"sma","close",self.sma_param_2)
        process_col(data,"sma","close",self.sma_param_3)
        process_col(data,"sma","close",self.sma_param_4)
        process_col(data,"ema","close",self.ema_param_1)
        process_col(data,"ema","close",self.ema_param_2)
        process_col(data,"ema","close",self.ema_param_3)
        process_col(data,"ema","close",self.ema_param_4)
        process_col(data,"hma","close",self.hma_param_1)
        process_col(data,"hma","close",self.hma_param_2)
        process_col(data,"hma","close",self.hma_param_3)
        process_col(data,"hma","close",self.hma_param_4)
        process_col(data,"macd",self.macd_param_1,self.macd_param_2,self.macd_param_3)
        process_col(data,"hamacd",self.hamacd_param_1,self.hamacd_param_2,self.hamacd_param_3)
        
        process_col(data,"batch_atr_stoploss",self.atr_stoploss_1_param_1,
                              self.atr_stoploss_1_param_2,self.atr_stoploss_1_param_3,
                              self.atr_stoploss_1_param_4,self.atr_smooth_param)
    def get_col_to_plot(self):
        plot1 = []
        plot2 = []
        plot3 = []
        plot4 = []
        plot5 = []
        
        
        plot1.append((self.ha_open,"lines"))
        plot1.append((self.ha_close,"lines"))
        plot1.append((self.ha_high,"lines"))
        plot1.append((self.ha_low,"lines"))
        plot1.append((self.renkoHigh,"lines"))
        plot1.append((self.renkoLow,"lines"))
        plot1.append((self.renko2High,"lines"))
        plot1.append((self.renko2Low,"lines"))
        plot1.append((self.senkouA,"lines"))
        plot1.append((self.senkouB,"lines"))
        plot1.append((self.senkouA2,"lines"))
        plot1.append((self.senkouB2,"lines"))
        plot1.append((self.atr_stoploss_1,"lines"))
        plot1.append((self.ema_1,"lines"))
        plot1.append((self.ema_2,"lines"))
        plot1.append((self.ema_3,"lines"))
        plot1.append((self.ema_4,"lines"))
        plot1.append((self.sma_shift_1,"lines"))
        plot1.append((self.sma_1,"lines"))
        plot1.append((self.sma_2,"lines"))
        plot1.append((self.sma_3,"lines"))
        plot1.append((self.sma_4,"lines"))
        plot1.append((self.hma_1,"lines"))
        plot1.append((self.hma_2,"lines"))
        plot1.append((self.hma_3,"lines"))
        plot1.append((self.hma_4,"lines"))
        plot1.append((self.bband_high,"lines"))
        plot1.append((self.bband_mid,"lines"))
        plot1.append((self.bband_low,"lines"))
        plot1.append((self.kijunsen,"lines"))
        plot1.append((self.ssl_up,"lines"))
        plot1.append((self.ssl_down,"lines"))
        plot2.append((self.ASH_bull,"lines"))
        plot2.append((self.ASH_bear,"lines"))
        plot3.append((self.waddah_bull,"lines"))
        plot3.append((self.waddah_explo,"lines"))
        plot3.append((self.waddah_bear,"lines"))
        plot3.append((self.waddah_dead,"lines"))
        
        plot4.append((self.stocharsi,"lines"))
        plot4.append((self.renkoCount,"lines"))
        plot4.append((self.renko2Count,"lines"))
        plot4.append((self.adx,"lines"))
        plot4.append((self.adx_sma,"lines"))
        plot5.append((self.hamacd,"lines"))
        plot5.append((self.hamacd_signal,"lines"))
        plot5.append((self.hamacd_hist,"lines"))
        plot5.append((self.rvi,"lines"))
        plot5.append((self.rvi_signal,"lines"))
        #plot5.append((self.rvi_hma,"lines"))
        #plot5.append((self.rvi_signal_hma,"lines"))
        plot5.append((self.macd,"lines"))
        plot5.append((self.macd_signal,"lines"))
        plot5.append((self.macd_hist,"lines"))
        return plot1,plot2,plot3,plot4,plot5
        
        

    
    def get_action(self, data, i, shares_can_buy, shares_can_sell, target, risk_percent, risk_atr_multiplier):
        import random
        action ="HOLD", 0, data
        
        if(i<100):
            return action
        ########################################################
        #Base Line 1
        '''
        self.atr = name_col("atr",self.atr_param_1)
        self.adx = name_col("adx",self.adx_param1)
        self.adx_sma = name_col("sma",self.v1_co1,self.adx_sma_param1)
        self.kijunsen = name_col("kijunsen",self.kijunsen_param)
        self.ssl_up = name_col("ssl_up",self.ssl_param_1)
        self.ssl_down = name_col("ssl_down",self.ssl_param_1)
        self.ASH_bull = name_col("ASH_bull",self.ash_param1,self.ash_param2)
        self.ASH_bear = name_col("ASH_bear",self.ash_param1,self.ash_param2)
        self.waddah_bull  = name_col("waddah_bull",self.waddah_param1,self.waddah_param2,self.waddah_param3,self.waddah_param4,self.waddah_param5,self.waddah_param6)
        self.waddah_bear = name_col("waddah_bear",self.waddah_param1,self.waddah_param2,self.waddah_param3,self.waddah_param4,self.waddah_param5,self.waddah_param6)
        self.waddah_explo = name_col("waddah_explo",self.waddah_param1,self.waddah_param2,self.waddah_param3,self.waddah_param4,self.waddah_param5,self.waddah_param6)
        self.waddah_dead = name_col("waddah_dead",self.waddah_param1,self.waddah_param2,self.waddah_param3,self.waddah_param4,self.waddah_param5,self.waddah_param6)
        self.rvi = name_col("rvi",self.rvi_param1)
        self.rvi_signal = name_col("rvi_signal",self.rvi_param1)
        self.close_sma = name_col("sma","close",self.sma_param)
        '''
        
        baseline=self.kijunsen
        atr = self.atr
        period = self.confirmation_delay
        baseline_signal=[-1]*period
        baseline_agree=[False]*period
        baseline_atr=[False]*period
        
        
        for p in range(0,period):
            if(
             (data["open"][i-p]<data[baseline][i-p]
              and data["close"][i-p]>data[baseline][i-p])
              or 
              (data["close"][i-(1+p)]<data[baseline][i-(1+p)]
              and data["close"][i-p]>data[baseline][i-p])
            ):
                baseline_signal[p] = 0
                
            if(data["close"][i-p]>data[baseline][i-p]):
                baseline_agree[p] = True
                
            if(data["close"][i-p]<data[baseline][i-p]+(data[atr][i-p])):
                baseline_atr[p] = True
        ########################################################
        
        '''
        
        c1_signal,c1_agree =  lookback(i, data, "close",self.ema_2, period)
        #c2_signal,c2_agree =  lookback(i, data, self.ASH_bull,self.ASH_bear, period)
        c4_signal,c4_agree =  lookback(i, data, self.hma_4,self.sma_4, period)
        c3_signal,c3_agree =  lookback(i, data, self.rvi,self.rvi_signal, period)
        c11_signal,c11_agree =  lookback(i, data, self.rvi,self.rvi_signal, period)
        
        c1_signal,c1_agree =  lookback(i, data, self.macd,self.macd_signal, period)
        c1_signal,c1_agree =  lookback(i, data, self.ema_1,self.ema_4, period)
        c2_signal,c2_agree =  lookback(i, data, self.ssl_up,self.ssl_down, period)
        c4_signal,c4_agree =  lookback(i, data, self.adx,self.hline_20, period)
        c1_signal,c1_agree =  lookback(i, data, "close",self.ema_3, period)
        '''
        #c1_signal,c1_agree =  lookback(i, data, self.ema_1,self.ema_2, period)
        #c3_signal,c3_agree =  lookback(i, data, "close",self.sma_1, period)
        #c4_signal,c4_agree =  lookback(i, data, self.ssl_up,self.ssl_down, period)
        #c4_signal,c4_agree =  lookback(i, data, self.adx,self.hline_20, period)
        #c2_signal,c2_agree =  lookback(i, data, self.ASH_bull,self.ASH_bear, period)
        #confirmation_signal_1
        #c1_signal,c1_agree =  lookback(i, data, "close",self.kijunsen, period)
        #c2_signal,c2_agree =  lookback(i, data, "close",self.atr_stoploss_1, period)
        #c3_signal,c3_agree =  lookback(i, data, self.hma_2,self.hma_3, period)
        #c6_signal,c6_agree =  lookback(i, data, self.hma_1,self.ema_1, period)
        #c5_signal,c5_agree =  lookback(i, data, self.waddah_bull,self.waddah_explo, period)
        #c2_signal,c2_agree =  lookback(i, data, self.ssl_up,self.ssl_down, period)
        #c1_signal,c1_agree =  lookback(i, data, self.macd,self.macd_signal, period)
        #c2_signal,c2_agree =  lookback(i, data, self.ema_1,self.sma_4, period)
        #c2_signal,c2_agree =  lookback(i, data, self.ssl_up,self.ssl_down, period)
        #c3_signal,c3_agree =  lookback(i, data, self.ASH_bull,self.ASH_bear, period)
        #v2_signal,v2_agree =  lookback(i, data, self.adx,self.hline_20, period)
        #v1_signal,v1_agree =  lookback(i, data, "close",self.atr_stoploss_1, period)
        #v2_signal,v2_agree =  lookback(i, data, self.adx,self.adx_sma, period)
        #ema_sma_signal,ema_sma_agree =  lookback(i, data, self.ema_1,self.sma_3, period)
        #close_sma_signal,close_sma_agree =  lookback(i, data, "close",self.sma_3, period)
        
        #c2_signal,c2_agree =  lookback(i, data, self.ASH_bull,self.ASH_bear, period)
        c4_signal,c4_agree =  lookback(i, data, self.hma_4,self.sma_4, period)
        c5_signal,c5_agree =  lookback(i, data, self.renkoHigh,self.renkoLow, period)
        c6_signal,c6_agree =  lookback(i, data, self.waddah_bull,self.waddah_explo, period)
        c7_signal,c7_agree =  lookback(i, data, self.waddah_bear,self.waddah_explo, period)
        c2_signal,c2_agree =  lookback(i, data, self.ssl_up,self.ssl_down, period)
        c4_signal,c4_agree =  lookback(i, data, self.adx,self.hline_20, period)
        
        c3_signal,c3_agree =  lookback(i, data, "close",self.kijunsen, period)
        c4_signal,c4_agree =  lookback(i, data, self.ema_1,self.sma_2, period)
        c1_signal,c1_agree =  lookback(i, data, self.sma_1,self.sma_shift_1, period)
        c1_signal,c1_agree =  lookback(i, data, "close",self.sma_shift_1, period)
        c1_signal,c1_agree =  lookback(i, data, "close",self.sma_shift_1, period)
        c1_signal,c1_agree =  lookback(i, data, "close",self.ema_1, period)
        c1_signal,c1_agree =  lookback(i, data, self.sma_3,self.sma_4, period)
        c2_signal,c2_agree =  lookback(i, data, self.ema_1,self.ema_3, period)
        c1_signal,c1_agree =  lookback(i, data, self.sma_3,self.sma_4, period)
        c3_signal,c3_agree =  lookback(i, data, self.ssl_up,self.ssl_down, period)
        
        ########################################################
        #Entry Strategies 
        
        standard_entry =0
        if(
            c3_signal[0] == 0
          ):
            standard_entry = 1
            
            
            
        TP_ATR = 2
        SL_ATR = 2
        TARGET_PERC = 1.5
        LOSS_PERC = 0.01
        #take_profit = ((data[atr][i]*TP_ATR)+data["close"][i])
        take_profit = (data["close"][i]/100.0)*(100.0+TARGET_PERC)
        #stop_loss = (data["close"][i]-(data[atr][i]*SL_ATR))
        stop_loss = (data["close"][i]/100.0)*(100.0-LOSS_PERC)
        
        if(shares_can_buy*data["close"][i]>0 and shares_can_sell==0):
            if(i>200 and
                (
                    standard_entry == 1
                )
            ):
                if(shares_can_sell==0):
                    #setup ATR trailing stoploss 
                    data = self.set_stoploss(data, i , 14, risk_atr_multiplier, 
                                             early_stop_profit=target ,
                                             stop_early_times=self.early_stop, danger_stop_atr=6)
                    data = self.set_tp_sl(data,i,take_profit=take_profit,stop_loss = stop_loss)
                action="BUY", data["close"][i], data            
                 
               
                
        ########################################################
        #exit_signal
        exit_signal1,exit_agree1 =  lookback(i, data, self.ssl_down,self.ssl_up, period)
        exit = 0
        if(
            i>200
            and
            exit_signal1[0] == 0
          ):
            exit = 1
                
                
        if(shares_can_sell>0):
            if(i>200
                and shares_can_sell > 0
                and exit ==1
            ):
                action = "SELL", data["close"][i], data
                data = self.set_tp_sl(data,i)
        
                
        ''' 
        if(shares_can_sell>0): 
            if(i>200
                and data["stoploss"][i]>min(data.low[i],data.close[i])
                and shares_can_sell > 0
            ):  
                action = "STOPPED", data["stoploss"][i], data
                data = self.set_tp_sl(data,i)
        '''
        
       
        if(shares_can_sell>0): 
            if(i>200
                and data["stoploss"][i]>data.close[i]
                and shares_can_sell > 0
            ):  
                action = "STOPPED", data["stoploss"][i], data
                data = self.set_tp_sl(data,i)
                
        
        

        '''    
        if(shares_can_sell>0): 
            if(i>200
                and max(data["stoploss"][i],data["stoploss_fix"][i])>min(data.low[i],data.close[i])
                and shares_can_sell > 0
               and False
            ):  
                action = "STOPPED", max(data["stoploss"][i],data["stoploss_fix"][i]), data
                data = self.set_tp_sl(data,i)
       
        
        '''    
                

                
        return action

    

In [0]:
# %%time
stock="WFC-PN"
stock="WFC-PY"
stock="BAC-PY"
stock="D05.SI"
stock="WFC-PT"
stock="GBPUSD=X"
stock="EURHUF=X"
stock="AMZN"
stock="GBPJPY=X"
stock="42W.SI"
stock="THB=X"
stock="EURUSD=X"
stock="MYR=X"
stock="RUB=X"
stock="PM"
stock="BACHF"
stock="EURCAD=X"
stock="EURHUF=X"
stock="GOOGL"
stock="THB=X"
stock="AAPL"
stock="NFLX"
stock="PHP=X"
stock="BHP"
stock="EURGBP=X"
stock="JPY=X"
stock="CHU"
stock="EURHUF=X"
stock="AUDUSD=X"
stock="EURSEK=X"
stock="GOOGL"
stock="EPD"
stock="EURJPY=X"
stock="AUDUSD=X"
stock="SGD=X"
stock="TSLA"
stock="EURUSD=X"
stock="D05.SI"

sdate='2007-01-01'
edate='2012-01-01'
strategy=Strategy1()
# stoploss, kijunsen, ema1, ema2, ssl, adx, hline



data = get_price_data(stock,datetime(2019,1,1),datetime(2020,1,1))
data.reset_index(drop=False, inplace=True) 
data["stock"] = stock
result = back_test(strategy, data, cash = 100000, transaction_fee =0.001, assets={},
                                   target=0.01, risk_percent = 0.02, risk_atr_multiplier =3)


Getting price data ...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:49: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


	profit: 1.57%	asset: 101573.77144943635
		win count: 1	loss count: 0
		win percent: 0.02653580296014511	loss percent: 0


In [0]:
from plotly.offline import iplot
import plotly.graph_objs as go
import IPython
from plotly.offline import init_notebook_mode
display(IPython.core.display.HTML('''<script src="/static/components/requirejs/require.js"></script>'''))
init_notebook_mode(connected=False)
fig = plot_chart(data, result,stock,strategy)
iplot(fig)


Output hidden; open in https://colab.research.google.com to view.